## Tune the hyperparameters of the peptide encoding model

## Imports

#### Standard pydata imports

This also creates a default `logger` for use in the notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from argparse import Namespace

try:
    import pyllars.logging_utils as logging_utils
    logger = logging_utils.get_ipython_logger()
    logger.setLevel('INFO')
except:
    pass

# standard pydata imports
import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pathlib
import seaborn as sns; sns.set(style='white', color_codes=True)
import tqdm
import yaml

INFO     : NumExpr defaulting to 4 threads.


#### Custom imports

In [3]:
import pyllars.collection_utils as collection_utils
import pyllars.mpl_utils as mpl_utils
import pyllars.pandas_utils as pd_utils

import pyllars.utils

import pepenc
import pepenc.data.data_utils as data_utils
import pepenc.models


from ray import tune

from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch

import ray

import ray._private.resource_spec
ray._private.resource_spec._autodetect_num_gpus = lambda :  1

#### Functions

## Start code

In [4]:
config = data_utils.load_sample_config()

#search_alg = BayesOptSearch(metric="validation_median_absolute_error", mode="min")
search_alg = HyperOptSearch(metric="validation_median_absolute_error", mode="min")

scheduler = ASHAScheduler(metric="validation_median_absolute_error", mode="min")

search_space = config.copy()
search_space['embedding_dim'] = tune.qlograndint(2*3, 2*8, 2)
search_space['hidden_dim'] = tune.qlograndint(2*3, 2*8, 2)
search_space['lstm_layers'] = tune.qrandint(2, 4)
search_space['lstm_dropout'] = tune.quniform(0.2, 0.8, 0.1)

search_space

{'name': 'SampleConfigModel',
 'embedding_dim': <ray.tune.sample.Integer at 0x7f4a74110d90>,
 'hidden_dim': <ray.tune.sample.Integer at 0x7f49b5c7e1f0>,
 'lstm_layers': <ray.tune.sample.Integer at 0x7f49b5c7e0a0>,
 'lstm_dropout': <ray.tune.sample.Float at 0x7f49b5c7e340>,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_lr': 0.01,
 'weight_decay': 0,
 'lr_patience': 3,
 'batch_size': 128,
 'max_sequence_length': 17,
 'num_instances': 20,
 'num_patience_epochs': 5,
 'num_training_epochs': 20,
 'training_set': '/prj/peptide-encoder/data/raw/sample-peptides.training.csv',
 'validation_set': '/prj/peptide-encoder/data/raw/sample-peptides.validation.csv',
 'test_set': '/prj/peptide-encoder/data/raw/sample-peptides.test.csv',
 'aa_encoding_map': '/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl',
 'out_dir': '/tmp/aa-encoder-trainer'}

In [5]:

# name of your experiment
name="my-pepenc-tune-exp"

analysis = tune.run(
    pepenc.models.PeptideEncoderLSTM,
    stop={"training_iteration": 10},
    config=search_space,
    checkpoint_freq=5,
    num_samples=20,
    resources_per_trial={"gpu": 0.4, "cpu": 1},
    scheduler=scheduler,
    search_alg=search_alg,
    
    name=name,

    # a directory where results are stored before being
    # sync'd to head node/cloud storage
    local_dir=f"/tmp/{name}",
    
    # sync our checkpoints via rsync
    # you don't have to pass an empty sync config - but we
    # do it here for clarity and comparison
    #sync_config=sync_config,

    # we'll keep the best five checkpoints at all times
    checkpoint_score_attr="validation_median_absolute_error",
    keep_checkpoints_num=5,

    # a very useful trick! this will resume from the last run specified by
    # sync_config (if one exists), otherwise it will start a new tuning run
    #resume="AUTO",
    resume=False
)

INFO     : Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.


Trial name,status,loc,aa_encoding_map,adam_beta1,adam_beta2,adam_lr,batch_size,embedding_dim,hidden_dim,lr_patience,lstm_dropout,lstm_layers,max_sequence_length,name,num_instances,num_patience_epochs,num_training_epochs,out_dir,test_set,training_set,validation_set,weight_decay
PeptideEncoderLSTM_bcf7c9f1,PENDING,,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,12,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0


2022-02-08 00:50:33,173	ERROR syncer.py:111 -- Log sync requires rsync to be installed.
(bundle_reservation_check_func pid=1777) /home/bmmalone/anaconda3/envs/torch-test/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
(bundle_reservation_check_func pid=1777)   warnings.warn(
(bundle_reservation_check_func pid=1776) /home/bmmalone/anaconda3/envs/torch-test/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMa

Trial name,status,loc,aa_encoding_map,adam_beta1,adam_beta2,adam_lr,batch_size,embedding_dim,hidden_dim,lr_patience,lstm_dropout,lstm_layers,max_sequence_length,name,num_instances,num_patience_epochs,num_training_epochs,out_dir,test_set,training_set,validation_set,weight_decay
PeptideEncoderLSTM_bcf7c9f1,RUNNING,172.24.170.184:1777,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,12,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0
PeptideEncoderLSTM_bd16c664,RUNNING,172.24.170.184:1776,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,12,8,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0
PeptideEncoderLSTM_bd2ae120,PENDING,,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,8,3,0.2,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0


Trial name,status,loc,aa_encoding_map,adam_beta1,adam_beta2,adam_lr,batch_size,embedding_dim,hidden_dim,lr_patience,lstm_dropout,lstm_layers,max_sequence_length,name,num_instances,num_patience_epochs,num_training_epochs,out_dir,test_set,training_set,validation_set,weight_decay
PeptideEncoderLSTM_bcf7c9f1,RUNNING,172.24.170.184:1777,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,12,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0
PeptideEncoderLSTM_bd16c664,RUNNING,172.24.170.184:1776,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,12,8,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0
PeptideEncoderLSTM_bd2ae120,PENDING,,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,8,3,0.2,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0


...

Result for PeptideEncoderLSTM_49c47c29:
  date: 2022-02-08_00-55-50
  done: false
  experiment_id: fa3b4de18356400da6120fb9838e94a5
  hostname: MSI
  iterations_since_restore: 9
  loss: 0.020118753705930904
  node_ip: 172.24.170.184
  pid: 2455
  time_since_restore: 58.58293914794922
  time_this_iter_s: 6.132416486740112
  time_total_s: 58.58293914794922
  timestamp: 1644278150
  timesteps_since_restore: 0
  training_iteration: 9
  trial_id: 49c47c29
  validation_mean_squared_error: 0.02196589671075344
  validation_median_absolute_error: 0.09418633580207825
  


Trial name,status,loc,aa_encoding_map,adam_beta1,adam_beta2,adam_lr,batch_size,embedding_dim,hidden_dim,lr_patience,lstm_dropout,lstm_layers,max_sequence_length,name,num_instances,num_patience_epochs,num_training_epochs,out_dir,test_set,training_set,validation_set,weight_decay,iter,total time (s),loss,validation_mean_squared_error,validation_median_absolute_error
PeptideEncoderLSTM_49c47c29,RUNNING,172.24.170.184:2455,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,8,3,0.3,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,9,58.5829,0.0201188,0.0219659,0.0941863
PeptideEncoderLSTM_bcf7c9f1,TERMINATED,172.24.170.184:1777,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,12,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,9.16647,0.0500697,0.541143,0.736876
PeptideEncoderLSTM_bd16c664,TERMINATED,172.24.170.184:1776,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,12,8,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,10,87.0826,0.0360849,0.250621,0.462099
PeptideEncoderLSTM_bd2ae120,TERMINATED,172.24.170.184:1778,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,8,3,0.2,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,10,78.7446,0.0203345,0.0208112,0.0930376
PeptideEncoderLSTM_c51d7eec,TERMINATED,172.24.170.184:1775,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,8,6,3,0.3,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,10,77.0588,0.0332293,0.0309994,0.112591
PeptideEncoderLSTM_f398749e,TERMINATED,172.24.170.184:1950,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,8,12,3,0.2,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,8.39395,0.0579295,0.157883,0.381132
PeptideEncoderLSTM_f7bbb5ea,TERMINATED,172.24.170.184:1982,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,10,3,0.4,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,7.71712,0.0581758,0.0655945,0.201352
PeptideEncoderLSTM_ff997a8d,TERMINATED,172.24.170.184:2013,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,10,3,0.6,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,8.16256,0.0523665,0.290005,0.525666
PeptideEncoderLSTM_06f4a4dd,TERMINATED,172.24.170.184:2051,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,8,3,0.4,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.

Result for PeptideEncoderLSTM_49c47c29:
  date: 2022-02-08_00-55-56
  done: true
  experiment_id: fa3b4de18356400da6120fb9838e94a5
  hostname: MSI
  iterations_since_restore: 10
  loss: 0.019620466507910713
  node_ip: 172.24.170.184
  pid: 2455
  time_since_restore: 64.77436399459839
  time_this_iter_s: 6.19142484664917
  time_total_s: 64.77436399459839
  timestamp: 1644278156
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 49c47c29
  validation_mean_squared_error: 0.0202947985380888
  validation_median_absolute_error: 0.09164425730705261
  


Trial name,status,loc,aa_encoding_map,adam_beta1,adam_beta2,adam_lr,batch_size,embedding_dim,hidden_dim,lr_patience,lstm_dropout,lstm_layers,max_sequence_length,name,num_instances,num_patience_epochs,num_training_epochs,out_dir,test_set,training_set,validation_set,weight_decay,iter,total time (s),loss,validation_mean_squared_error,validation_median_absolute_error
PeptideEncoderLSTM_bcf7c9f1,TERMINATED,172.24.170.184:1777,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,12,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,9.16647,0.0500697,0.541143,0.736876
PeptideEncoderLSTM_bd16c664,TERMINATED,172.24.170.184:1776,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,12,8,3,0.7,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,10,87.0826,0.0360849,0.250621,0.462099
PeptideEncoderLSTM_bd2ae120,TERMINATED,172.24.170.184:1778,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,8,3,0.2,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,10,78.7446,0.0203345,0.0208112,0.0930376
PeptideEncoderLSTM_c51d7eec,TERMINATED,172.24.170.184:1775,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,8,6,3,0.3,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,10,77.0588,0.0332293,0.0309994,0.112591
PeptideEncoderLSTM_f398749e,TERMINATED,172.24.170.184:1950,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,8,12,3,0.2,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,8.39395,0.0579295,0.157883,0.381132
PeptideEncoderLSTM_f7bbb5ea,TERMINATED,172.24.170.184:1982,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,10,3,0.4,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,7.71712,0.0581758,0.0655945,0.201352
PeptideEncoderLSTM_ff997a8d,TERMINATED,172.24.170.184:2013,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,14,10,3,0.6,2,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,8.16256,0.0523665,0.290005,0.525666
PeptideEncoderLSTM_06f4a4dd,TERMINATED,172.24.170.184:2051,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,6,8,3,0.4,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.test.csv,/prj/peptide-encoder/data/raw/sample-peptides.training.csv,/prj/peptide-encoder/data/raw/sample-peptides.validation.csv,0,1,8.364,0.0693733,0.194681,0.409166
PeptideEncoderLSTM_0de4c7de,TERMINATED,172.24.170.184:2083,/prj/peptide-encoder/data/intermediate/oh-aa-encoding-map.jpkl,0.9,0.999,0.01,128,10,8,3,0.4,3,17,SampleConfigModel,20,5,20,/tmp/aa-encoder-trainer,/prj/peptide-encoder/data/raw/sample-peptides.t

2022-02-08 00:55:56,412	INFO tune.py:626 -- Total run time: 323.54 seconds (323.29 seconds for the tuning loop).


In [6]:
cols = [
    'config.embedding_dim', 'config.hidden_dim', 'config.lstm_layers', 'config.lstm_dropout', 'config.vocabulary_size',
    'loss', 'validation_mean_squared_error', 'validation_median_absolute_error', 'training_iteration'
]

analysis.results_df[cols].sort_values('validation_median_absolute_error')

/home/bmmalone/anaconda3/envs/torch-test/lib/python3.9/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,config.embedding_dim,config.hidden_dim,config.lstm_layers,config.lstm_dropout,config.vocabulary_size,loss,validation_mean_squared_error,validation_median_absolute_error,training_iteration
trial_id,,,,,,,,,
49c47c29,14,8,2,0.3,21,0.019620,0.020295,0.091644,10
bd2ae120,6,8,2,0.2,21,0.020335,0.020811,0.093038,10
c51d7eec,8,6,2,0.3,21,0.033229,0.030999,0.112591,10
2ad37b0e,6,6,2,0.4,21,0.032216,0.035392,0.119530,4
399ed68f,6,6,2,0.3,21,0.032465,0.044965,0.146751,4
42c42a0b,12,6,2,0.5,21,0.037087,0.063316,0.166004,4
f7bbb5ea,6,10,2,0.4,21,0.058176,0.065594,0.201352,1
2d6e5d9e,10,16,2,0.4,21,0.041948,0.135624,0.350321,1
f398749e,8,12,3,0.2,21,0.057929,0.157883,0.381132,1


In [9]:
analysis.results_df.columns

Index(['loss', 'validation_mean_squared_error',
       'validation_median_absolute_error', 'done', 'timesteps_total',
       'episodes_total', 'training_iteration', 'experiment_id', 'date',
       'timestamp', 'time_this_iter_s', 'time_total_s', 'pid', 'hostname',
       'node_ip', 'time_since_restore', 'timesteps_since_restore',
       'iterations_since_restore', 'experiment_tag', 'config.name',
       'config.embedding_dim', 'config.hidden_dim', 'config.lstm_layers',
       'config.lstm_dropout', 'config.adam_beta1', 'config.adam_beta2',
       'config.adam_lr', 'config.weight_decay', 'config.lr_patience',
       'config.batch_size', 'config.max_sequence_length',
       'config.num_instances', 'config.num_patience_epochs',
       'config.num_training_epochs', 'config.training_set',
       'config.validation_set', 'config.test_set', 'config.aa_encoding_map',
       'config.out_dir', 'config.vocabulary_size'],
      dtype='object')

In [7]:
# Gets best trial based on max accuracy across all training iterations.
best_trial = analysis.get_best_trial(metric="validation_median_absolute_error", mode="min", scope="all") 

# Gets best checkpoint for trial based on accuracy.
best_checkpoint = analysis.get_best_checkpoint(best_trial, metric="validation_median_absolute_error", mode='min')

best_checkpoint

'/tmp/my-pepenc-tune-exp/my-pepenc-tune-exp/PeptideEncoderLSTM_49c47c29_20_aa_encoding_map=_prj_peptide-encoder_data_intermediate_oh-aa-encoding-map.jpkl,adam_beta1=0.9,adam__2022-02-08_00-54-29/checkpoint_000010/checkpoint.pt'

In [8]:
m = pepenc.models.PeptideEncoderLSTM.load(best_checkpoint)
m._val_step()

INFO     : Reading config file
INFO     : [torch_utils.get_device] creating device: 'cuda'
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1981/1981 [00:00<00:00, 163938.92it/s]
INFO     : [SampleConfigModel] restoring the network
INFO     : [SampleConfigModel] restoring the optimizer


{'validation_mean_squared_error': 2.085783e-02,
 'validation_median_absolute_error': 0.08977765}

In [16]:
type(analysis)

ray.tune.analysis.experiment_analysis.ExperimentAnalysis

In [11]:
a = ray.tune.ExperimentAnalysis("/tmp/my-pepenc-tune-exp/")

2022-02-08 01:12:36,468	INFO experiment_analysis.py:673 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


In [14]:

best_trial = a.get_best_trial(metric="validation_median_absolute_error", mode="min", scope="all") 

# Gets best checkpoint for trial based on accuracy.
bc = a.get_best_checkpoint(best_trial, metric="validation_median_absolute_error", mode='min')
bc

'/tmp/my-pepenc-tune-exp/my-pepenc-tune-exp/PeptideEncoderLSTM_49c47c29_20_aa_encoding_map=_prj_peptide-encoder_data_intermediate_oh-aa-encoding-map.jpkl,adam_beta1=0.9,adam__2022-02-08_00-54-29/checkpoint_000010/checkpoint.pt'

In [15]:
m = pepenc.models.PeptideEncoderLSTM.load(bc)
m._val_step()

INFO     : Reading config file
INFO     : [torch_utils.get_device] creating device: 'cuda'
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1981/1981 [00:00<00:00, 204371.22it/s]
INFO     : [SampleConfigModel] restoring the network
INFO     : [SampleConfigModel] restoring the optimizer


{'validation_mean_squared_error': 1.9561063e-02,
 'validation_median_absolute_error': 0.08693158}